In [1]:
import import_ipynb
import random
from scipy.linalg import expm
from netsquid.components.models.qerrormodels import QuantumErrorModel
from netsquid.qubits import qubitapi as qapi
from netsquid.qubits.operators import Operator, Z, I

In [2]:
class IonPhotonDephasingNoiseModel(QuantumErrorModel):
    """
    Model for applying collective dephasing noise to qubit(s) on an ion trap.
    """

    def __init__(self, coherence_time):
        super().__init__()
        self.required_properties.append('dephasing_rate')
        self.coherence_time = coherence_time

    def error_operation(self, qubits, delta_time=10e-9, **properties):
        """Noisy quantum operation to apply to qubits.

        Parameters
        ----------
        qubits : tuple of :obj:`~netsquid.qubits.qubit.Qubit`
            Qubits to apply noise to.
        delta_time : float, optional
            Time qubit has spent on component [ns].

        """
        dephasing_rate = properties['dephasing_rate']
        coherence_time = self.coherence_time
        if coherence_time != 0.:
            rotation_matrix = expm(1j * dephasing_rate * delta_time / coherence_time * Z.arr)
        else:
            rotation_matrix = I
        rotation_operator = Operator(name='rotation_operator', matrix=rotation_matrix,
                                     description='collective dephasing operator, rotation rate has been sampled')
        for qubit in qubits:
            qapi.operate(qubit, rotation_operator)
